# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [73]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [74]:
weather_metadata = "output_data\my_weather_data.csv"

# Read the weather data and the study results
weather_data = pd.read_csv(weather_metadata)

# Combine the data into a single dataset
weatherdf = pd.DataFrame(weather_data)
weatherdf.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mehamn,71.0357,27.8492,47.10,77,84,6.60,NO,1590602400
1,1,Bulgan,48.8125,103.5347,49.95,84,100,11.39,MN,1590602400
2,2,Tasiilaq,65.6145,-37.6368,38.89,71,66,3.44,GL,1590602400
3,3,Cape Town,-33.9258,18.4232,58.48,70,83,16.58,ZA,1590602400
4,4,Avarua,-21.2078,-159.7750,74.68,77,17,6.73,CK,1590602400


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [77]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)
# Store 'Lat' and 'Lng' into  locations 
locations = weatherdf[["Latitude", "Longitude"]].astype(float)

#Store humidity 
hum_rate = weatherdf["Humidity"].astype(float)

In [78]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer)
fig
#plt.savefig("vacationheatmap.png")

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [79]:
#A max temperature lower than 80 degrees but higher than 70.
first = weatherdf.loc[weatherdf["Max Temp"]<80,:]
second = first.loc[weatherdf["Max Temp"]>70,:]
second.head()

#Wind speed less than 10 mph.
third = second.loc[second["Wind Speed"]<10,:]
third

#Zero cloudiness.
fourth = third.loc[third["Cloudiness"]==0,:]
fourth.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,44,Caravelas,-17.7125,-39.2481,79.38,57,0,7.92,BR,1590602400
120,120,Vangaindrano,-23.3500,47.6000,71.91,86,0,8.28,MG,1590602400
121,121,Diego de Almagro,-26.3667,-70.0500,78.46,23,0,8.37,CL,1590602400
126,126,Cangzhou,38.3167,116.8667,70.02,28,0,6.69,CN,1590602400
128,128,Pishin,30.5803,66.9961,78.76,16,0,0.60,PK,1590602400


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [80]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(fourth)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Latitude']
hotel_df['Lng'] = fifth['Longitude']
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
44,Caravelas,BR,-17.7125,-39.2481,
120,Vangaindrano,MG,-23.3500,47.6000,
121,Diego de Almagro,CL,-26.3667,-70.0500,
126,Cangzhou,CN,38.3167,116.8667,
128,Pishin,PK,30.5803,66.9961,


In [63]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get hotel type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 44: Caravelas.
Closest hotel to Caravelas at -17.7125 , -39.2481 is Nacional Gás.
------------
Retrieving Results for Index 120: Vangaindrano.
Closest hotel to Vangaindrano at -23.35 , 47.6 is CEG Ampasy, Vangaindrano.
------------
Retrieving Results for Index 121: Diego de Almagro.
Closest hotel to Diego de Almagro at -26.3667 , -70.05 is Radio Bahia Diego de Almagro.
------------
Retrieving Results for Index 126: Cangzhou.
Closest hotel to Cangzhou at 38.3167 , 116.8667 is Cangzhou Xinhua Pricing Administration.
------------
Retrieving Results for Index 128: Pishin.
Closest hotel to Pishin at 30.5803 , 66.9961 is Pishin Post Office.
------------
Retrieving Results for Index 144: Miracema.
Closest hotel to Miracema at -21.4122 , -42.1967 is Câmara de Dirigentes Lojistas de Miracema.
------------
Retrieving Results for Index 147: Yuty.
Closest hotel to Yuty at -26.6 , -56.25 is Supervisión Educativa Región 1 Caazapá.
------------
Retrieving Results for Inde

In [68]:
hotel_df.head()


,City,Country,Lat,Lng,Hotel Name
44,Caravelas,BR,-17.7125,-39.2481,Nacional Gás
120,Vangaindrano,MG,-23.3500,47.6000,"CEG Ampasy, Vangaindrano"
121,Diego de Almagro,CL,-26.3667,-70.0500,Radio Bahia Diego de Almagro
126,Cangzhou,CN,38.3167,116.8667,Cangzhou Xinhua Pricing Administration
128,Pishin,PK,30.5803,66.9961,Pishin Post Office


In [69]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

In [82]:
# Add marker layer ontop of heat map

fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

marker_layer = gmaps.marker_layer(locations2, hover_text='', label='', info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("vacationheatmapwithhotels.png")

fig

# Display Map

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>